# Show More Details and Enhance the Contrast

In [103]:

from PIL import Image, ImageFilter , ImageEnhance
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import sys
from matplotlib import cm

In [104]:
img = Image.open('apple2.JPG')
enc_img = img.filter(ImageFilter.DETAIL)
# enc_img.show()



img = Image.open('apple2.JPG')
img_con = ImageEnhance.Contrast(enc_img)
img_con.enhance(1.5).show("90% more contrast")

# crop the image 

In [41]:
img = Image.open('apple1.JPG')
#print(img.mode)

box = (250, 90, 900, 700)
crop_img = img.crop(box) #left, upper, right, lower
crop_img.show()

In [115]:
def removeBackground(path):
    img = cv.imread(path, cv.IMREAD_UNCHANGED)

    original = img.copy()

    l = int(max(5, 6))
    u = int(min(6, 6))

    ed = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    edges = cv.GaussianBlur(img, (51, 3), 3)
    edges = cv.cvtColor(edges, cv.COLOR_BGR2GRAY)
    edges = cv.Canny(edges, l, u)

    _,thresh = cv.threshold(edges, 0, 255, cv.THRESH_BINARY  + cv.THRESH_OTSU)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3, 3))
    mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=0)


    data = mask.tolist()
    sys.setrecursionlimit(10**8)
    for i in  range(len(data)):
        for j in  range(len(data[i])):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
        for j in  range(len(data[i])-1, -1, -1):
            if data[i][j] !=  255:
                data[i][j] =  -1
            else:
                break
    image = np.array(data)
    image[image !=  -1] =  255
    image[image ==  -1] =  0

    mask = np.array(image, np.uint8)

    result = cv.bitwise_and(original, original, mask=mask)
    result[mask ==  0] =  0
    cv.imwrite('bg.png', result)
    print(result.dtype)


    img = Image.open('bg.png')
    img.convert("RGBA")
    print(img)

    datas = img.getdata()

    newData = []
    for item in datas:
        if item[0] ==  255  and item[1] ==  255  and item[2] ==  255:
            newData.append((0, 0, 0, 255))
        else:
            newData.append(item)

    img.putdata(newData)
    return img